In [14]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from match_image_data import convert_csv_2_dict, load_images_from_folder, get_drone_state



DATAFILE_DRONE = "/home/matthijs/paparazzi/data/AE4317_2019_datasets/cyberzoo_aggressive_flight/20190121-144708.csv"
IMAGES_FOLDER = "/home/matthijs/paparazzi/data/AE4317_2019_datasets/cyberzoo_aggressive_flight/20190121-144646/"
VIDEO_FILE = "/home/matthijs/robotics_q3/mav/cyberzoo.mp4"
VIDEO_FILE_SIM = "/home/matthijs/paparazzi/videos/vlc-record-2023-03-03-11h21m26s-rtp_5000.sdp-.avi"
VIDEO_ALL_OBSTACLES = "/home/matthijs/paparazzi/videos/vlc-record-2023-03-10-10h06m31s-rtp_5000.sdp-.avi"
VIDEO_ORANGE_AVOID = "/home/matthijs/paparazzi/videos/vlc-record-2023-03-10-09h59m06s-rtp_5000.sdp-.avi"
SHOW_CV2_IMG = False
SHOW_VIDEO = True

drone_state_dict = convert_csv_2_dict(DATAFILE_DRONE)
images_cyberzoo_dict = load_images_from_folder(IMAGES_FOLDER)
images_cyberzoo = images_cyberzoo_dict["images"]
images_cyberzoo_stamp = images_cyberzoo_dict["timestamps"]

AttributeError: module 'numpy.random' has no attribute 'BitGenerator'

In [13]:
!pip install opencv-python
!pip install pandas

     |████████████████████████████████| 12.2 MB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 14.3 MB/s eta 0:00:01    |██████████████████▎             | 9.9 MB 61.2 MB/s eta 0:00:01
     |████████████████████████████████| 502 kB 18.8 MB/s eta 0:00:01


In [3]:
def show_img(img, gray_scale=False, img_name="img"):
    if SHOW_CV2_IMG:
        cv2.imshow(img_name, img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        if gray_scale:
            cmap = "gray"
        else:
            cmap = None
        plt.imshow(img, cmap=cmap)


In [4]:
def rgb_2_gray(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray


In [13]:
# This Optical flow implementation yields to way better results
import time
from collections import deque


def draw_flow(img, flow, step=16):

    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T

    lines = np.vstack([x, y, x-fx, y-fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(img_bgr, lines, 0, (0, 255, 0))

    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(img_bgr, (x1, y1), 1, (0, 255, 0), -1)

    return img_bgr


def draw_hsv(flow):

    h, w = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]

    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)

    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return bgr

def draw_flow_sides(flow_left, flow_right):
    img_flow_sides = np.zeros((200, 50))
    img_flow_sides[:100, :] = flow_left
    img_flow_sides[100:, :] = flow_right
    return img_flow_sides




if SHOW_VIDEO:

    cap = cv2.VideoCapture(VIDEO_ORANGE_AVOID)

    suc, prev = cap.read()

    prev = prev[160:360,95:145]
    prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    #prevgray = canny_edge_detection(prevgray)
    flow_left_queue = deque(maxlen=3)
    flow_right_queue = deque(maxlen=3)
    while True:

        suc, img = cap.read()
        if img is None:
            continue 
        img = img[160:360,95:145]
        
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #gray = canny_edge_detection(gray)
        # start time to calculate FPS
        start = time.time()


        flow_l = cv2.calcOpticalFlowFarneback(prevgray[:100,:], gray[:100,:], None, 0.5, 3, 15, 3, 5, 1.2, 0)
        flow_r = cv2.calcOpticalFlowFarneback(prevgray[100:,:], gray[100:,:], None, 0.5, 3, 15, 3, 5, 1.2, 0)
        flow_left = np.sum(np.linalg.norm(flow_l, axis=2))
        flow_right = np.sum(np.linalg.norm(flow_r, axis=2))
        print(flow_left.shape, flow_right.shape)
        flow_left_queue.append(flow_left)
        flow_right_queue.append(flow_right)
        sum_flow_left = np.sum(flow_left_queue)
        sum_flow_right = np.sum(flow_right_queue)
        print(sum_flow_left, sum_flow_right)
        flow_left_norm = sum_flow_left / max(sum_flow_left, sum_flow_right)
        flow_right_norm = sum_flow_right / max(sum_flow_left, sum_flow_right)

        # print(flow_left_norm, flow_right_norm)
        prevgray = gray


        # End time
        end = time.time()
        # calculate the FPS for current frame detection
        fps = 1 / (end-start)

        print(f"{fps:.2f} FPS")

        #cv2.imshow('flow', draw_flow(gray, flow))
        cv2.imshow('flow sides', draw_flow_sides(flow_left_norm, flow_right_norm))
        #cv2.imshow('flow HSV', draw_hsv(flow))


        key = cv2.waitKey(1000)
        if key == ord('q'):
            break


    cap.release()
    cv2.destroyAllWindows()


() ()
2605.4631 2470.5425
665.66 FPS
() ()
4795.7495 5320.7305
533.02 FPS
() ()
8141.8174 8288.392
508.15 FPS
() ()
7851.1523 8263.01
545.21 FPS
() ()
8295.564 8205.986
517.30 FPS
() ()
7308.2197 8005.0513
638.11 FPS
() ()
8649.148 7335.7163
468.69 FPS
() ()
8256.6 6532.745
203.62 FPS
() ()
7850.732 6277.215
186.97 FPS
() ()
6946.402 7107.7085
573.15 FPS
() ()
7160.758 7591.0864
247.93 FPS
() ()
7143.908 7331.7686
524.55 FPS
() ()
7356.828 7416.761
384.23 FPS
() ()
7968.5415 8217.333
307.61 FPS
() ()
8810.588 7844.0483
451.10 FPS
() ()
7437.6855 7417.384
203.69 FPS
() ()
7712.827 6109.21
649.88 FPS
() ()
8110.886 7189.0293
599.96 FPS
() ()
8676.063 7630.9185
563.83 FPS
() ()
7280.931 8325.059
407.17 FPS
() ()
6065.9937 7577.3213
205.28 FPS
() ()
6061.8105 7191.138
294.73 FPS
() ()
28609.215 18947.852
443.56 FPS
() ()
53453.18 32431.871
273.73 FPS
() ()
71410.36 38918.508
294.48 FPS
() ()
60257.727 32609.605
296.50 FPS
() ()
41242.957 17762.781
326.33 FPS
() ()
31554.557 9598.1455
203.5

: 

: 

In [23]:
images_folder = "/home/matthijs/Downloads/AE4317_2019_datasets/cyberzoo_poles_panels_mats/20190121-142935"

from os import walk

images = []
for (dirpath, dirnames, filenames) in walk(images_folder):
    images.extend(filenames)
    break
images=np.sort(images)
print(images)

['11549407.jpg' '11649419.jpg' '11749459.jpg' '11882755.jpg'
 '11982761.jpg' '12082792.jpg' '12216115.jpg' '12316094.jpg'
 '12416095.jpg' '12549415.jpg' '12649423.jpg' '12782745.jpg'
 '12882752.jpg' '12982755.jpg' '13082756.jpg' '13216093.jpg'
 '13316087.jpg' '13449408.jpg' '13549415.jpg' '13682741.jpg'
 '13782751.jpg' '13916085.jpg' '14049404.jpg' '14149410.jpg'
 '14249412.jpg' '14382739.jpg' '14482742.jpg' '14582742.jpg'
 '14682742.jpg' '14816068.jpg' '14916074.jpg' '15049398.jpg'
 '15149402.jpg' '15282740.jpg' '15382735.jpg' '15516063.jpg'
 '15616070.jpg' '15749403.jpg' '15849435.jpg' '15949399.jpg'
 '16082728.jpg' '16216056.jpg' '16349382.jpg' '16449408.jpg'
 '16582719.jpg' '16682719.jpg' '16816056.jpg' '16916058.jpg'
 '17049381.jpg' '17149386.jpg' '17282711.jpg' '17382727.jpg'
 '17516045.jpg' '17616053.jpg' '17749382.jpg' '17849382.jpg'
 '17982711.jpg' '18082715.jpg' '18216042.jpg' '18349394.jpg'
 '18449372.jpg' '18582706.jpg' '18682709.jpg' '18816048.jpg'
 '18916045.jpg' '1904936

In [31]:
import cv2
from collections import deque
flow_left_queue = deque(maxlen=3)
flow_right_queue = deque(maxlen=3)
flow_middle_queue = deque(maxlen=3)
WIDTH = 30
HEIGHT = 140
def create_windows(whole_window, width, height):
    left_window = whole_window[:width, :int(height/2)]
    middle_window = whole_window[:width, int(height/4):int(3*height/4)]
    right_window = whole_window[:width, :height]
    return left_window, middle_window, right_window

def scale_window(whole_window, width, height):
    img_shape = img.shape
    width_img = img_shape[1]
    height_img = img_shape[0]
    scaled_window = whole_window[int(width_img - width/2):int(width_img - width/2), int(height_img - height/2):int(height_img - height/2)]
    return scaled_window

def calculate_output_flow(flow):
    return np.linalg.norm(flow, axis=2)
    
for i, img_f in enumerate(images):
    img = cv2.imread(f"{images_folder}/{img_f}")
    cv2.imshow("img", img)
    scaled_img = scale_window(img, 50, 50)
    img_grey = cv2.cvtColor(scaled_img, cv2.COLOR_BGR2GRAY)
    if i==0:
        prev_grey = np.copy(img_grey)
        continue

    flow = cv2.calcOpticalFlowFarneback(prev_grey, img_grey, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    flow_left, flow_middle, flow_right = create_windows()
    flow_left_queue.append(calculate_output_flow(flow_left))
    flow_right_queue.append(calculate_output_flow(flow_right))
    flow_middle_queue.append(calculate_output_flow(flow_middle))
    prev_grey = np.copy(img_grey)
    key = cv2.waitKey(50)
    if key == ord('q'):
        break   
cv2.destroyAllWindows()

# cap = cv2.VideoCapture(VIDEO_ORANGE_AVOID)

#     suc, prev = cap.read()

#     prev = prev[160:360,95:145]
#     prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
#     #prevgray = canny_edge_detection(prevgray)
#     flow_left_queue = deque(maxlen=3)
#     flow_right_queue = deque(maxlen=3)
#     while True:

#         suc, img = cap.read()
#         if img is None:
#             continue 
#         img = img[160:360,95:145]
        
#         gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         #gray = canny_edge_detection(gray)
#         # start time to calculate FPS
#         start = time.time()


#         flow_l = cv2.calcOpticalFlowFarneback(prevgray[:100,:], gray[:100,:], None, 0.5, 3, 15, 3, 5, 1.2, 0)
#         flow_r = cv2.calcOpticalFlowFarneback(prevgray[100:,:], gray[100:,:], None, 0.5, 3, 15, 3, 5, 1.2, 0)
#         flow_left = np.sum(np.linalg.norm(flow_l, axis=2))
#         flow_right = np.sum(np.linalg.norm(flow_r, axis=2))
#         print(flow_left.shape, flow_right.shape)
#         flow_left_queue.append(flow_left)
#         flow_right_queue.append(flow_right)
#         sum_flow_left = np.sum(flow_left_queue)
#         sum_flow_right = np.sum(flow_right_queue)
#         print(sum_flow_left, sum_flow_right)
#         flow_left_norm = sum_flow_left / max(sum_flow_left, sum_flow_right)
#         flow_right_norm = sum_flow_right / max(sum_flow_left, sum_flow_right)

#         # print(flow_left_norm, flow_right_norm)
#         prevgray = gray


#         # End time
#         end = time.time()
#         # calculate the FPS for current frame detection
#         fps = 1 / (end-start)

#         print(f"{fps:.2f} FPS")

#         #cv2.imshow('flow', draw_flow(gray, flow))
#         cv2.imshow('flow sides', draw_flow_sides(flow_left_norm, flow_right_norm))
#         #cv2.imshow('flow HSV', draw_hsv(flow))


#         key = cv2.waitKey(1000)
#         if key == ord('q'):
#             break


#     cap.release()
#     cv2.destroyAllWindows()


error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
